# Introduction

Auther: Yingding Wang\
Created: 15.11.2023

this notebook introduces load a pdf file from s3 bucket with boto3 and ByteIO stream with `pypdf.PdfReader`.

* Boto3 client doc: https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/s3.html#S3.Client.list_objects
* S3 bucket doesn't have folder, but prefix https://stackoverflow.com/questions/51303609/python-recursive-glob-in-s3/51303951#51303951
* Boto3 latest pypi version https://pypi.org/project/boto3/

### show ENV variables of the pod
```python
import os
print(os.environ)
```
or
```
!env
```

### PdfReader with BytesIO
* https://stackoverflow.com/questions/48373967/issue-with-pypdf2-and-decoding-pdf-file-from-s3



In [1]:
import sys

In [2]:
# Installing collected packages: jmespath, botocore, s3transfer, boto3
# !{sys.executable} -m pip install --user --upgrade boto3==1.29.0

In [12]:
import os
import boto3
bucket_name="scivias-medreports"
session = boto3.session.Session(
    aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
)
#s3 = session.resource('s3', endpoint_url = os.environ.get('S3_ENDPOINT'), verify=False)
s3 = session.resource('s3', endpoint_url = os.environ.get('S3_ENDPOINT'))
bucket = s3.Bucket(bucket_name)

In [16]:
#type(s3)

In [4]:
#bucket_items = []
#for obj in bucket.objects.all():
#    bucket_items.append(obj.key)
#    # print(obj)

In [5]:
bucket.objects.filter(Prefix="KK-SCIVIAS")

s3.Bucket.objectsCollection(s3.Bucket(name='scivias-medreports'), s3.ObjectSummary)

## s3 has no regex ListObjects

s3 list objects with prefix only
https://stackoverflow.com/questions/62379936/searching-for-keys-in-a-s3-bucket-with-prefix-suffix-or-regex


In [6]:
# pattern="KK-SCIVIAS-*.pdf"
prefix="KK-SCIVIAS"
batch_max  = 2
# for obj in bucket.objects.filter
bucket_items = []
for obj in bucket.objects.filter(Prefix=prefix):
    bucket_items.append(obj.key)
    # print(obj)

In [9]:
len(bucket_items)

243

In [7]:
from io import BytesIO
from pypdf import PdfReader

In [8]:
idx = 0
item = bucket_items[idx]

In [9]:
obj = s3.Object(bucket_name, item)

In [10]:
fs = obj.get()['Body'].read()

In [11]:
type(fs)

bytes

In [14]:
reader = PdfReader(BytesIO(fs))

In [15]:
content_raw_str = "".join([page.extract_text() for page in reader.pages])

In [16]:
len(content_raw_str)

7611